# Kickstarting the Martian Greenhouse Effect
### A Final Project for Physics 361 by Morgan Baxter and Alex Brovender



For a variety of reasons, including proximity to the sun and atmospheric and regolith composition, Mars is the number one candidate body in the solar system for future human colonization. 

<div>
<img src="https://upload.wikimedia.org/wikipedia/commons/6/6a/PIA22546-Mars-AnnualCO2ice-N%26SPoles-20180806.gif" width="500"/>
</div>
$\textbf{Figure 1:}$ Extents of north (left) and south (right) polar CO2 ice during a martian year

$\textbf{Source:}$ NASA/JPL-caltech